In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# ---------------------------
# 1) Load data
# ---------------------------
# Expecting a CSV with columns: text, label
# label: 1=spam/phish, 0=ham
df = pd.read_csv("emails.csv")  # <-- you will create this from corpus
df = df.dropna(subset=["text", "label"])

# ---------------------------
# 2) Clean text
# ---------------------------
def clean_email(text: str) -> str:
    text = str(text)
    text = re.sub(r"<[^>]+>", " ", text)          # remove HTML
    text = re.sub(r"http\S+|www\.\S+", " URL ", text)  # replace links
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["clean_text"] = df["text"].apply(clean_email)

X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"], df["label"],
    test_size=0.2, random_state=42, stratify=df["label"]
)

# ---------------------------
# 3) TF-IDF vectorization
# ---------------------------
tfidf = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    max_features=30000
)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

# ---------------------------
# 4) Model
# ---------------------------
model = LogisticRegression(max_iter=2000)
model.fit(X_train_vec, y_train)

# ---------------------------
# 5) Evaluation
# ---------------------------
pred = model.predict(X_test_vec)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))

# ---------------------------
# 6) Top words (interpretability)
# ---------------------------
feature_names = tfidf.get_feature_names_out()
coefs = model.coef_[0]

top_spam = coefs.argsort()[-20:][::-1]
top_ham = coefs.argsort()[:20]

print("\nTop Spam/Phish Indicators:")
print([feature_names[i] for i in top_spam])

print("\nTop Ham Indicators:")
print([feature_names[i] for i in top_ham])


Confusion Matrix:
 [[1 0]
 [0 2]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Top Spam/Phish Indicators:
['link', 'account', 'password', 'iphone clicking', 'clicking link', 'clicking', 'free iphone', 'free', 'iphone', 'win', 'win free', 'offer', 'offer buy', 'time', 'time offer', '50 discount', '50', 'buy', 'buy 50', 'limited time']

Top Ham Indicators:
['attached', 'project', 'let', 'invoice attached', 'invoice', 'attached review', 'review', 'deadline', 'reminder', 'reminder submit', 'submit', 'submit assignment', 'assignment', 'assignment deadline', 'completed', 'completed successfully', 'modules', 'project update', 'modules completed', 'successfully']
